#DeepVoice3: Single-speaker text-to-speech demo

In this notebook, you can try DeepVoice3-based single-speaker text-to-speech (en) using a model trained on [LJSpeech dataset](https://keithito.com/LJ-Speech-Dataset/). The notebook is supposed to be executed on [Google colab](https://colab.research.google.com) so you don't have to setup your machines locally.

**Estimated time to complete**: 5 miniutes.

- Code: https://github.com/r9y9/deepvoice3_pytorch
- Audio samples: https://r9y9.github.io/deepvoice3_pytorch/

## Setup

### Install dependencies

In [0]:
import os
from os.path import exists, join, expanduser

# Clone
name = "deepvoice3_pytorch"
if not exists(name):
  ! git clone https://github.com/r9y9/$name

In [2]:
# Change working directory to the project dir 
%cd deepvoice3_pytorch/

!git checkout 7a10ac6763eda92595e257543494b6a95f64229b --quiet

# Install dependencices
!pip install -q -e '.[bin]'

/content/deepvoice3_pytorch


In [3]:
%pylab inline
! pip install -q librosa nltk
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize


import torch
from PIL import Image
import numpy as np
import librosa
import librosa.display
import IPython
from scipy.io import wavfile
import random
import pandas as pd
from IPython.display import Audio
# need this for English text processing frontend
import nltk
! python -m nltk.downloader cmudict

Populating the interactive namespace from numpy and matplotlib
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


### Download a pre-trained model

In [0]:
preset = "20180505_deepvoice3_ljspeech.json"
checkpoint_path = "20180505_deepvoice3_checkpoint_step000640000.pth"

In [0]:
if not exists(preset):
  !curl -O -L "https://www.dropbox.com/s/0ck82unm0bo0rxd/20180505_deepvoice3_ljspeech.json"
if not exists(checkpoint_path):
  !curl -O -L "https://www.dropbox.com/s/5ucl9remrwy5oeg/20180505_deepvoice3_checkpoint_step000640000.pth"

## Synthesis

### Setup hyper parameters

In [6]:
import hparams
import json

    
# Load parameters from preset
with open(preset) as f:
  hparams.hparams.parse_json(f.read())
  
# Inject frontend text processor
import synthesis
import train
from deepvoice3_pytorch import frontend
synthesis._frontend = getattr(frontend, "en")
train._frontend =  getattr(frontend, "en")

# alisesa
fs = hparams.hparams.sample_rate
hop_length = hparams.hparams.hop_size

W0704 15:46:43.172587 139816328099712 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### Define utility functions

In [0]:
def read_wavfile(filename):
    _, samples = wavfile.read(filename)
    samples = samples.astype(np.float64)
    #print(samples.shape)
    return samples

def get_spectrogram(samples): 
    # TODO - try out different ways:
    # https://librosa.github.io/librosa/generated/librosa.display.specshow.html
    D = librosa.amplitude_to_db(np.abs(librosa.stft(samples)), ref=np.max)
    #print(D.shape)
    return D
  
def plot_spectrogram(D,filename,display = False):    
    # set display = True to display the graph
    fig=plt.figure(figsize=(60,40))
    plt.subplot(4, 2, 2)
    librosa.display.specshow(D, y_axis='mel')
    plt.axis('off')
    plt.savefig('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/spectrograms/'+filename +  '.jpg',bbox_inches='tight',transparent=True,pad_inches=0.0)
    if not display:
      plt.close(fig)
    #plt.show()

def tts(model, text, p=0, speaker_id=None, fast=True, figures=False, display=False):
  from synthesis import tts as _tts
  waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
  D = get_spectrogram(waveform)
  #print(D.shape)
  if figures:
      plot_spectrogram(D,text,display)
      IPython.display.display(Audio(waveform, rate=fs))
  return D
   

# def visualize(alignment, spectrogram):
#   label_fontsize = 16
#   figure(figsize=(16,16))

#   subplot(2,1,1)
#   imshow(alignment.T, aspect="auto", origin="lower", interpolation=None)
#   xlabel("Decoder timestamp", fontsize=label_fontsize)
#   ylabel("Encoder timestamp", fontsize=label_fontsize)
#   colorbar()

#   subplot(2,1,2)
#   librosa.display.specshow(spectrogram.T, sr=fs, 
#                            hop_length=hop_length, x_axis="time", y_axis="linear")
#   xlabel("Time", fontsize=label_fontsize)
#   ylabel("Hz", fontsize=label_fontsize)
#   tight_layout()
#   colorbar()

### Load the model checkpoint

In [8]:
from train import build_model
from train import restore_parts, load_checkpoint

model = build_model()
model = load_checkpoint(checkpoint_path, model, None, True)

Load checkpoint from: 20180505_deepvoice3_checkpoint_step000640000.pth


## Training

### Dataset

In [9]:
%cd /content

/content


In [10]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [0]:
# real_dataset = [] # List of file paths to LJSpeech .wav files.
# for i in range(1,9):
#   for filename in os.listdir('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs'+str(i)):
#     real_dataset.append('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs'+str(i)+'/'+filename)
# # random.shuffle(real_dataset)
# real_dataset.sort()

In [0]:
real_dataset=[]
for filename in os.listdir('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs'):
    real_dataset.append('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/'+filename)
# random.shuffle(real_dataset)
real_dataset.sort()

In [0]:
# global df = pd.DataFrame()
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata1.csv',sep='|')
sentences = df[df.columns[1]]
sentences = sentences.tolist()
# print(sentences[:5])
# sentences=sentences[1:]
# #fake-dataset
# print(sentences[:5])

####Correction

In [14]:
#for i in range(0,20):
#   df =df.drop([0])
#   df.to_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata1.csv',sep='|',index=False)
#   df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata1.csv',sep='|')
  print(df.head())
  print(real_dataset[:5])

    file name  ...                                      Same sentence
0  LJ004-0216  ...  The most noticeable of the improvements introd...
1  LJ004-0217  ...  The old haphazard system, by which meat was is...
2  LJ004-0218  ...  a week's allowance at a time, was abolished, a...
3  LJ004-0219  ...  The diet was now ample. It consisted of a poun...
4  LJ004-0220  ...  for breakfast a pint of gruel; for dinner half...

[5 rows x 3 columns]
['/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0216.wav', '/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0217.wav', '/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0218.wav', '/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0219.wav', '/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0220.wav']


In [0]:
# # !rm /content/gdrive/My\ Drive/Colab\ Notebooks/LJSpeech-1.1/metadata.csv
#  !cp /content/gdrive/My\ Drive/LJSpeech-1.1/metadata1.csv  /content/gdrive/My\ Drive/Colab\ Notebooks/LJSpeech-1.1/metadata1.csv

In [0]:
#  %cd /content/gdrive/My\ Drive/LJSpeech-1.1/wavs1
#  !cp -v LJ001-00* /content/gdrive/My\ Drive/Colab\ Notebooks/LJSpeech-1.1/wavs

In [0]:
# !cp /content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ002-0081.wav /content/gdrive/My\ Drive/Colab\ Notebooks/LJSpeech-1.1/wavs/LJ002-0081.wav

In [0]:
# !rm /content/gdrive/My\ Drive/Colab\ Notebooks/generator.pth
# !rm /content/gdrive/My\ Drive/Colab\ Notebooks/discriminator.pth

In [19]:
print(real_dataset[0]==("/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/"+str(df.iloc[0,:][0])+".wav"))
print(str(real_dataset[0]))
print("/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/"+str(df.iloc[0,:][0])+".wav")

True
/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0216.wav
/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/LJ004-0216.wav


### Get data

In [0]:
real_idx=0
fake_idx=0
width = 250 # Fixed width of the waveform to be given as input to the model - needs tuning

In [0]:
def get_real():
  """Returns a REAL speech sample from the LJSpeech dataset as a PIL Image.
     The width is fixed at 250 units - if changed, needs to be changed for both get_real() and get_fake() functions"""
  global real_idx
  waveform = read_wavfile(real_dataset[real_idx])
  D = get_spectrogram(waveform)
  if(D.shape[1]>=width):
    plot_spectrogram(D[:,:width],'temp_real')
#     spec_img = Image.open('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/spectrograms/temp.jpg')
#     return spec_img
  else:
    plot_spectrogram(D[:,:],'temp_real')

  os.remove(real_dataset[real_idx])
  real_idx += 1

In [0]:
def get_fake():
  """Returns a FAKE speech sample as a PIL Image by passing a random sentence from the LJSpeech dataset through DeepVoice3
     The width is fixed at 250 units - if changed, needs to be changed for both get_real() and get_fake() functions"""
  global fake_idx
  D = tts(model,sentences[fake_idx])
  fake_idx += 1
  if(D.shape[1]>=width):
    plot_spectrogram(D[:,:width],'tempfake')
#     spec_img = Image.open('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/spectrograms/temp.jpg')
#     return spec_img
  else:
    plot_spectrogram(D[:,:],'tempfake')

  df1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata1.csv',sep='|')

  df1=df1.iloc[1:,:]
  df1.to_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata1.csv',sep='|',index=False)
 

In [0]:
# df1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata.csv',sep='|')
# sentences1 = df1[df1.columns[1]]
# sentences1 = sentences1.tolist()
# print(sentences1[:5])
# df1 = df1.iloc[1:,:]
# print(df1.head())
# df1.to_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata.csv',sep='|',index=False)
# df2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/metadata.csv',sep='|')
# sentences2 = df2[df2.columns[2]]
# sentences2 = sentences2.tolist()
# print(sentences2[:5])


### GAN Model

In [23]:
from __future__ import print_function
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [0]:
# Root directory for dataset
dataroot = "/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 1

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 1

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [0]:

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [0]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [0]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [28]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [30]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [0]:
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [0]:
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 
# print (gpu.memoryUtil*100)

In [33]:
netG.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/generator.pth'))
netD.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/discriminator.pth'))



IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [34]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    dataset_size = min(len(sentences),len(real_dataset))
    if ((real_dataset[real_idx]==("/content/gdrive/My Drive/Colab Notebooks/LJSpeech-1.1/wavs/"+str(df.iloc[0,:][0])+".wav"))):

      for c in range(0,dataset_size):
        get_real()
        get_fake()

        # We can use an image folder dataset the way we have it setup.
        # Create the dataset
        dataset = dset.ImageFolder(root=dataroot,
                                   transform=transforms.Compose([
                                       transforms.Resize((image_size,image_size)),
                                       transforms.ToTensor(),
                                   ]))

        # Create the dataloader
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                 shuffle=True, num_workers=workers)

        # For each batch in the dataloader
        for i,data in enumerate(dataloader,0):

            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
          if i==0:         
            netD.zero_grad()
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            #b_size = 10
            label = torch.full((b_size,), real_label, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

          if (i==1):
            print(data[0].shape)
            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake_noise = netG(noise)
            fake_deepvoice=data[0].to(device)
            fake = fake_noise + fake_deepvoice
            label.fill_(fake_label)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Add the gradients from the all-real and all-fake batches
            errD = errD_real + errD_fake
            # Update D
            optimizerD.step()


    #           state1 = {
    #               'epoch': c,
    #               'state_dict': netD.state_dict(),
    #               'optimizer': optimizerD.state_dict(),
    #           }
            torch.save(netD.state_dict(),'/content/gdrive/My Drive/Colab Notebooks/discriminator.pth')



              ############################
              # (2) Update G network: maximize log(D(G(z)))
              ###########################
            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake).view(-1)
            # Calculate G's loss based on this output

            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizerG.step()

    #           state2 = {
    #               'epoch': c,
    #               'state_dict': netG.state_dict(),
    #               'optimizer': optimizerG.state_dict(),
    #           }
            torch.save(netG.state_dict(),'/content/gdrive/My Drive/Colab Notebooks/generator.pth')



            # Output training stats
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                      % (c, dataset_size, i+1, len(dataloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_losses.append(errD.item())

    #         if (gpu.memoryUtil*100==80):
    #           os.system('pkill -i -f "ipykernel_launcher"')
          # Check how the generator is doing by saving G's output on fixed_noise
    #           if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
    #               with torch.no_grad():
    #                   fake = netG(fixed_noise).detach().cpu()
    #               img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

    #           iters += 1


Starting Training Loop...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([1, 3, 64, 64])
[0/12015][2/2]	Loss_D: 1.2532	Loss_G: 0.9573	D(x): 0.5350	D(G(z)): 0.4662 / 0.3839
torch.Size([1, 3, 64, 64])
[1/12015][2/2]	Loss_D: 1.5729	Loss_G: 0.3067	D(x): 0.3982	D(G(z)): 0.4791 / 0.7359
torch.Size([1, 3, 64, 64])
[2/12015][2/2]	Loss_D: 1.4805	Loss_G: 0.9422	D(x): 0.7247	D(G(z)): 0.6860 / 0.3898
torch.Size([1, 3, 64, 64])
[3/12015][2/2]	Loss_D: 1.2493	Loss_G: 1.3147	D(x): 0.5679	D(G(z)): 0.4952 / 0.2686
torch.Size([1, 3, 64, 64])
[4/12015][2/2]	Loss_D: 1.6488	Loss_G: 0.6970	D(x): 0.2751	D(G(z)): 0.3010 / 0.4981
torch.Size([1, 3, 64, 64])
[5/12015][2/2]	Loss_D: 1.5143	Loss_G: 0.4571	D(x): 0.4472	D(G(z)): 0.5082 / 0.6331
torch.Size([1, 3, 64, 64])
[6/12015][2/2]	Loss_D: 1.4621	Loss_G: 0.6529	D(x): 0.5908	D(G(z)): 0.6078 / 0.5205
torch.Size([1, 3, 64, 64])
[7/12015][2/2]	Loss_D: 1.2188	Loss_G: 0.9768	D(x): 0.5709	D(G(z)): 0.4823 / 0.3765
torch.Size([1, 3, 64, 64])
[8/12015][2/2]	Loss_D: 1.2521	Loss_G: 0.8152	D(x): 0.4798	D(G(z)): 0.4041 / 0.4426
torch.Size

RuntimeError: ignored

In [0]:
!ps aux | grep ipykernel_launcher

In [0]:
!pkill -f ipykernel_launcher